# Speech latent feature discovery for multi-label forensic profiling

Project for IWBF 2018

----

**Objective:** Find latent representations in speech that can predict forensic aspects of human.


## Update ~12/11/2017

1. Write-up:

    https://www.overleaf.com/12702004wrrmdybmpqsq#/48448185/

2. Implementation
    1. Data: TIMIT, Alcohol, (SRE)
    2. Model
        ![model](./write-up/figs/model.png)  
    3. Current attributes: speaker-id, gender, dialect, age, height, drunk
    4. [Code](./src)
    
3. Final poster for convex

## Update ~12/18/2017

1. Experiments
    1. Currently using latent codes from autoencoder to do multi-label classification and regression.
    2. It works well on certain tasks (gender, dialect, height), but not well on other tasks (id, age).
    3. Including more datasets (SRE, Yandong's face data)


|Label | Reconstruction Loss ($l_2^2$) | ID Acc (%) | Gender Acc (%) | Dialect Acc (%) | Age MSE | Height MSE |
|:----|----:|---:|---:|---:|----:|---:|
| Train | 0.8565 | 75.15 | 96.15 | 89.40 | 4.5411 | 0.1634 |
| Test | 2.2059 | 66.15 | 92.15 | 83.17  | 5.1525 | 0.2353 |

 
2. Writing paper

3. Todo next week
    1. Finish draft of the paper.
    2. Improve the model.
    